# LSTM Regression Model

In [1]:
from river_torch.regression import RollingRegressor
from river import metrics, compose, preprocessing
from river.datasets import Bikes
import torch
from tqdm import tqdm
from river_torch.utils import SequentialLSTM

In [2]:
def build_torch_lstm_regressor(n_features, hidden_size=1):
    net = torch.nn.Sequential(
        SequentialLSTM(input_size=n_features, hidden_size=hidden_size, num_layers=1),
        torch.nn.Linear(hidden_size, 10),
        torch.nn.Linear(10, 1)
    )
    return net

In [3]:
dataset = Bikes()
metric = metrics.MAE()
optimizer_fn = torch.optim.SGD

model_pipeline = compose.Select('clouds', 'humidity', 'pressure', 'temperature', 'wind')
model_pipeline |= preprocessing.StandardScaler()
model_pipeline |= RollingRegressor(
    build_fn=build_torch_lstm_regressor,
    loss_fn='mse',
    optimizer_fn=torch.optim.SGD,
    window_size=20,
    lr=1e-2,
    hidden_size=1,
    num_layers=1,
    append_predict=True,
)

In [4]:
for x, y in tqdm(dataset.take(5000)):
    y_pred = model_pipeline.predict_one(x)
    metric = metric.update(y_true=y, y_pred=y_pred)
    model_pipeline = model_pipeline.learn_one(x=x, y=y)
print(f'MAE: {metric.get():.2f}')

0it [00:00, ?it/s]C:\Users\kulbach\Documents\projects\IncrementalLearning\river-torch\river_torch\regression\rolling_regressor.py:125: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  loss = self.loss_fn(y_pred, y)
5000it [00:25, 198.34it/s]

MAE: 4.31
